In [1]:
#for dev purpose
import os
USERNAME = os.getenv('USERNAME').lower()
new_pull = True

In [2]:
# import python packages
import sys
if sys.version_info[1]==12:
    os.add_dll_directory(r'C:/Program Files/Python/312/Lib/site-packages/clidriver/bin')
else:
    pass
import pandas as pd
import datetime as dt
import time
import warnings
from snp_query_box import DsnpHelperFunction, snpQueries
from snp_query_box.dsnp_transform import raceTransform
warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", None)
print(f"User: {USERNAME} Started creating race_df in the shared storage")

User: a845514 Started creating race_df in the shared storage


In [3]:
# set the starting time
start_time = time.time()

In [4]:
# calculate the dates
today = dt.datetime.today()
today_str = today.strftime('%Y-%m-%d')
last_date_of_last_month_str = DsnpHelperFunction.last_date_of_last_month(today_str, output_type="string")
last_date_of_last_month = DsnpHelperFunction.last_date_of_last_month(today_str)

reporting_end_date = last_date_of_last_month_str
print(reporting_end_date)

2025-05-31


In [5]:
# set the storage and output path
storage_path = r'//mbip/medicarepBI/Projects/COE/DSNP/dsnp_data_storage/monthly_flat_files'
output_path = f'{storage_path}/{reporting_end_date}'

isExist = os.path.exists(output_path)
print(output_path)
if not isExist:
    # Create a new directory because it does not exist
     os.makedirs(output_path)
     print("The new directory is created!")
else:
      print("The folder already exist")
print("Output files will sit in the folder above.")

//mbip/medicarepBI/Projects/COE/DSNP/dsnp_data_storage/monthly_flat_files/2025-05-31
The folder already exist
Output files will sit in the folder above.


In [6]:
#locate data dump folder
if new_pull:
    pull_race = snpQueries.pull_race()
    pull_race.to_parquet(f"{output_path}/pull_race.parquet")
else:
    try:
        pull_race = pd.read_parquet(f"{output_path}/pull_race.parquet")

    except:
        pull_race = snpQueries.pull_race()
        pull_race.to_parquet(f"{output_path}/pull_race.parquet")

100%|██████████| 20241024/20241024 [00:51<00:00, 396515.33it/s]


In [7]:
# output the top rows and size
race_df = raceTransform.get_race_df(pull_race)
race_df = race_df.sort_values("Race_ID").drop_duplicates(subset = 'MBI', keep='first')
race_df.to_parquet(f'{output_path}/race_df.parquet')
print(race_df.shape)
print(race_df.head(3))

(20240188, 4)
         Race_ID  StarsMember_ID           MBI Race_Code
5733133      0.0          784455  3RT0TJ2NY42     Others
5575584      0.0         3935878   3P98K20PE21    Others
3718117      0.0         3576547   2PU3KQ8VC85    Others


In [8]:
# output the process time
print("Process time -- %s seconds" % (time.time() - start_time))

Process time -- 233.95109391212463 seconds
